# Model 2: Revisió de comentaris

This notebook loads some Excels with comments not labeled and evaluate them




In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy.util import minibatch, compounding
from spacy_langdetect import LanguageDetector
import pycld2 as cld2

In [4]:
# Test if a file has the correct format to load his information

def validateSheet(data):

    cols = data.columns
    if (cols.size > 5):
        if (('Codi pregunta' in cols) or ('Comentari' in cols) or ('Tipus incidencia' in cols)):
            if (('Codi pregunta' in cols) and ('Comentari' in cols) and ('Tipus incidencia' in cols)):
                print (Fore.GREEN + " " * 8  + "VALID")
                return True
            else:
                print (Fore.RED + " " * 8 + "WARNING (see column names)")
                print (cols)
                return False
        else:
            print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
            return False
    else:
        print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
        return False


In [5]:
# Reorganize information to be exported in preprocessed files

def process_file (data_ori, file):
# folder = "17-18"
# file = "assignatura_professor_comentaris_LS_17-18_1s.xlsx"
# sheet = "Hoja1"
# xlsx = pd.ExcelFile(pathori+"/"+folder+"/"+file)
# data_ori = pd.read_excel(xlsx, sheet)

#    print(Fore.BLACK + "Process_file")
#    data_ori = data_ori[data_ori["Comentari"].notna]
    data_ori["Fitxer"] = file
    data_ori["TipusPregunta"] = data_ori["Codi pregunta"].str[0]
    data_ori["TipusIncidencia"] = data_ori["Tipus incidencia"]

    # Obtain languages of comments    
    data_ori["Idiomas"] = data_ori.apply(lambda row: detect_language(row["Comentari"]), axis=1)
    data_ori["IdiomaSpacy"] = data_ori.apply(lambda row: row["Idiomas"][0], axis=1)
    data_ori["IdiomaPyCld2"] = data_ori.apply(lambda row: row["Idiomas"][1], axis=1)
    data_ori["Idioma"] = data_ori.apply(lambda row: row["Idiomas"][2], axis=1)

    #data_res = data_ori[["Curs","Fitxer","Codi estudi","Codi assign.","TipusPregunta","TipusIncidencia","Comentari","IdiomaSpacy","IdiomaPyCld2","Idioma"]]
    data_res = data_ori[["Curs","Codi estudi","Codi assign.","TipusPregunta","TipusIncidencia","Comentari","Idioma"]]
    
    #data_res = data_res.dropna(subset="Comentari")
    data_res = data_res[data_res["Comentari"].notnull()]
    
    if debug:
#        print("Comments:"  + data_res.shape[0])
        display(data_res[data_res["TipusIncidencia"].notnull()].head())
    
    return data_res

In [6]:
# Detects the language of a comment. Compares results from Spacy and PyCld2

def detect_language(comment):
    
    lang_spacy = nlp(comment)._.language["language"]
    isReliable, textBytesFound, details = cld2.detect(comment.encode('utf-8', 'replace'), 
                                                    isPlainText = True, bestEffort = True, returnVectors=False)
    lang_pycld2 = details[0][1]
    
    language = lang_spacy
    if (lang_spacy != "ca") & (lang_pycld2 == "ca"):
        language = "ca"

    if (lang_spacy != "ca") & (lang_spacy != "es") & (lang_pycld2 == "es"):
        language = "es"
        
    if (lang_spacy == "en") & (lang_pycld2 != "ca") & (lang_pycld2 != "es") & (lang_pycld2 != "en"):
        language = "ca"
        
    if (debug == 2):
        print (comment, ", ", lang_spacy, ", ", lang_pycld2, ", ", language)
     
 #   language = lang_spacy
    
 #   return language
    return [lang_spacy, lang_pycld2, language]

In [7]:
pathori = "../data/original/"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/model2/"
debug = 1

In [8]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("ca_fasttext_wiki")
# ca_fasttext_wiki
# es_core_news_sm
# en_core_web_sm

nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

/home/david/.local/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'ca_fasttext_wiki' (0.0.2) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [33]:
def predict_comment(comment):
    doc = nlp2(comment)
    cats = doc.cats
    
    #return cats
    #return cats["POSITIVE"]

    if cats["POSITIVE"] > 0.5:
        return "Comentari problemàtic"
    else:
        return ""
    

In [36]:
folder = "19-20/1S/"
file = "assignatura_professor_19_20_graus_eps_1s.xls"

url = pathori + folder + file
print("File: ", url)

sheet = ""
xlsx = pd.ExcelFile(url)

for sheet in xlsx.sheet_names:
    if debug:
        print (Fore.BLACK + "Sheet:  " + sheet)
    data = pd.read_excel(xlsx, sheet)
    if validateSheet(data):
#                print ("Processing file: " + url)
        data_res = process_file(data, file)
        print("Comments: ", data_res.shape[0])
        display(data_res.sample(5))
        
        # Falta loop languages
        
        language = "ca"
        
        data_lang = data_res[data_res["Idioma"]=="ca"]
        print("Language:", language, ", Data lang: ", data_lang.shape[0])
        
        # test the saved model
        print("Loading model from", pathmodel+ language)
        #nlp2 = spacy.load(pathmodel +  language)
        
        data_lang["TipusIncidencia"] = data_lang.apply(lambda row: predict_comment(row["Comentari"]), axis=1)
        data_true = data_lang[data_lang["TipusIncidencia"]!=""]
        print("Data true: ", data_true.shape[0])
        display(data_lang[data_lang["TipusIncidencia"]!=""])
        
        

File:  ../data/original/19-20/1S/assignatura_professor_19_20_graus_eps_1s.xls
Sheet:  Sheet0
        VALID


,Curs,Codi estudi,Codi assign.,TipusPregunta,TipusIncidencia,Comentari,Idioma


Comments:  610


,Curs,Codi estudi,Codi assign.,TipusPregunta,TipusIncidencia,Comentari,Idioma
288,2019-20,G52,102000,A,NaN,Crec que programacio és una assignatura esenci...,ca
91,2019-20,G58,102172,P,NaN,Ya voldriem tots tenir mes professors com ella.,ca
140,2019-20,G58,102178,A,NaN,Entenc el mètode d'imposar-nos l'anglès però p...,ca
351,2019-20,TC0,102104,A,NaN,Estaria bé replantejar la distribució de les h...,ca
494,2019-20,G52,102010,A,NaN,"Penso que tenim molt bons professors, es preoc...",ca


Language: ca , Data lang:  495
Loading model from ../data/models/model2/ca
Els dilluns arriba 10 minuts tard a l'hora de pralab 1
{'POSITIVE': 0.9999954700469971, 'NEGATIVE': 4.573910700855777e-06}
Molt bona professora, el temari s'entén bé i ens respon a tots els dubtes.
{'POSITIVE': 5.2290703024482355e-05, 'NEGATIVE': 0.9999476671218872}
Crec que s'haurie de tenir una mica mes en compte el que fa un i laltre per a poder acurar mes en els exercicis i practiques
{'POSITIVE': 0.0005032864282839, 'NEGATIVE': 0.9994966983795166}
Potser es podrien fer mes exercicis a classe per reforçar l'aprenentatge
{'POSITIVE': 0.10705336928367615, 'NEGATIVE': 0.8929466605186462}
El temari de les classes de Teoria i les de practica no esta coordinat, ja que a les classes de “Pralab” es donaba temari mes avançat que a Teoria i feia perdre el fil del temari
{'POSITIVE': 5.1476217777235433e-05, 'NEGATIVE': 0.9999485015869141}
Data true:  151


<ipython-input-36-bdabf383dacc>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lang["TipusIncidencia"] = data_lang.apply(lambda row: predict_comment(row["Comentari"]), axis=1)


,Curs,Codi estudi,Codi assign.,TipusPregunta,TipusIncidencia,Comentari,Idioma
1,2019-20,G52,102002,P,Comentari problemàtic,Els dilluns arriba 10 minuts tard a l'hora de ...,ca
6,2019-20,G52,102002,P,Comentari problemàtic,Em sembla que és donen massa exemples i poca e...,ca
8,2019-20,G52,102002,A,Comentari problemàtic,Hem sembla molt correcte aquesta assignatura j...,ca
10,2019-20,G52,102002,P,Comentari problemàtic,"Hem sembla una bona professora explicant, s'en...",ca
12,2019-20,G52,102002,P,Comentari problemàtic,"Molt dinamica, s'esforça molt per que els alum...",ca
...,...,...,...,...,...,...,...
603,2019-20,GOI,102491,P,Comentari problemàtic,Li manca una mica més de direcció en la estruc...,ca
604,2019-20,GOI,102491,P,Comentari problemàtic,"Que haig de dir del Grau, sempre dispossat a a...",ca
605,2019-20,DIA,102000,A,Comentari problemàtic,Examens amb ordenador amb temps limitat per a ...,ca
608,2019-20,TC0,102118,P,Comentari problemàtic,"Un tracta mes suavitzat, contribuiria millors ...",ca
